In [9]:
import os, sys
import pandas as pd

# Get the current working directory
current_directory = os.getcwd()

# Move one level up
parent_directory = os.path.dirname(current_directory)

In [10]:
sys.path.append(parent_directory)

In [12]:
from data_ingestion.iex_data import *

In [14]:
get_processed_data('rtm')

Data updated upto:  12-01-2024


,datetime,mcp_rtm,clearedvolume_rtm,pb_rtm,sb_rtm,diff_sb_pb_rtm
0,2020-07-01 00:00:00,1749.01,658.90,676.5,2126.5,-1450.0
1,2020-07-01 00:15:00,999.98,642.50,646.6,2249.6,-1603.0
2,2020-07-01 00:30:00,2005.45,637.70,1150.1,2137.4,-987.3
3,2020-07-01 00:45:00,2003.55,638.00,1108.1,2150.7,-1042.6
4,2020-07-01 01:00:00,2500.08,672.70,1291.7,1924.7,-633.0
...,...,...,...,...,...,...
123839,2024-01-11 23:45:00,3190.98,2666.94,4002.2,10447.5,-6445.3
123840,2024-01-12 00:00:00,2920.12,2821.20,3827.4,11944.7,-8117.3
123841,2024-01-12 00:15:00,3050.95,2556.27,3785.0,12153.0,-8368.0
123842,2024-01-12 00:30:00,2927.99,2324.40,3469.1,7380.8,-3911.7


In [7]:
pd.read_pickle('/Users/amanbhatt/Desktop/price-forecast-iex/data/raw/rtm')

,_id,hour,time_block,date,purchase_bid,sell_bid,mcv,mcp,cleared_volume,final_sch_volume,vol_loss_rtm_curt,session_id,updated_at,created_at
0,659edb454085442c8b1fca22,1,00:00-00:15,11-01-2024,3351.8,10898.3,2176.00,2880.02,None,2176.00,None,1,2024-01-10T18:00:37.299000Z,2024-01-10T18:00:37.299000Z
1,659edb454085442c8b1fca23,1,00:15-00:30,11-01-2024,3340.2,10879.2,2046.01,2927.71,None,2046.01,None,1,2024-01-10T18:00:37.300000Z,2024-01-10T18:00:37.300000Z
2,659ee26e675fe5012a6a769e,1,00:30-00:45,11-01-2024,3511.2,5847.6,2117.40,2880.39,None,2117.40,None,2,2024-01-10T18:31:10.193000Z,2024-01-10T18:31:10.193000Z
3,659ee26e675fe5012a6a769f,1,00:45-01:00,11-01-2024,3347.6,6086.1,2063.54,2810.41,None,2063.54,None,2,2024-01-10T18:31:10.194000Z,2024-01-10T18:31:10.194000Z
4,659ee94226811d55ad5516d2,2,01:00-01:15,11-01-2024,3304.3,11276.2,2598.80,2460.29,None,2598.80,None,3,2024-01-10T19:00:18.279000Z,2024-01-10T19:00:18.279000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,65a01ed54786a0127c1b99e3,24,23:15-23:30,11-01-2024,3919.7,9983.9,2702.60,3391.40,None,2702.60,None,47,2024-01-11T17:01:09.220000Z,2024-01-11T17:01:09.220000Z
94,65a0263530735000087ffe02,24,23:30-23:45,11-01-2024,3943.9,10292.8,2605.81,3200.10,None,2605.81,None,48,2024-01-11T17:32:37.627000Z,2024-01-11T17:32:37.627000Z
95,65a0263530735000087ffe03,24,23:45-24:00,11-01-2024,4002.2,10447.5,2666.94,3190.98,None,2666.94,None,48,2024-01-11T17:32:37.628000Z,2024-01-11T17:32:37.628000Z
96,65a02ccece481467ec422622,1,00:00-00:15,12-01-2024,3827.4,11944.7,2821.20,2920.12,None,2821.20,None,1,2024-01-11T18:00:46.977000Z,2024-01-11T18:00:46.977000Z
